In [1]:
# !pip install bs4

In [2]:
# Import libraries
import requests
import urllib.request
import time
from bs4 import BeautifulSoup

In [4]:
# Set the URL you want to webscrape from
url = 'https://www.linkedin.com/jobs/search?keywords=Software%20Engineer&location=Berlin%2C%20Berlin%2C%20Germany&geoId=&trk=homepage-jobseeker_jobs-search-bar_search-submit&redirect=false&position=1&pageNum=0'

In [5]:
# Connect to the URL
response = requests.get(url)

# Parse HTML and save to BeautifulSoup object¶
soup = BeautifulSoup(response.text, "html.parser")

In [6]:
jobs = soup.findAll("li", attrs={"class":"result-card job-result-card result-card--with-hover-state"})
job_ids = [job["data-id"] for job in jobs]

In [9]:
for job_id in job_ids:
    url = """
    https://www.linkedin.com/jobs/search?keywords=Software%2BEngineer&location=Berlin%2C%2BBerlin%2C%2BGermany&trk=homepage-jobseeker_jobs-search-bar_search-submit&currentJobId={}&position=2&pageNum=0
    """.format(job_id)
#     print(url)

In [10]:

# for job_id in job_ids:
#     url = """
#     https://www.linkedin.com/jobs/search?keywords=Software%2BEngineer
#     &location=Berlin%2C%2BBerlin%2C%2BGermany
#     &trk=homepage-jobseeker_jobs-search-bar_search-submit
#     &currentJobId={}
#     &position=2
#     &pageNum=0
#     """.format(job_id)
#     # Connect to the URL
#     response = requests.get(url)

#     # Parse HTML and save to BeautifulSoup object¶
#     soup = BeautifulSoup(response.text, "html.parser")
#     print(soup)

# FOUND THAT LINKEDIN HAS THIS JOBS-GUEST API ;)

In [11]:


url = "https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/2428456471"
# Connect to the URL
response = requests.get(url)

# Parse HTML and save to BeautifulSoup object¶
soup = BeautifulSoup(response.text, "html.parser")

In [12]:
soup.find("section",attrs={"class":"description"}).text

'DescriptionSoftware Development Engineer Graduate - 2021 (Berlin) f/m/dDo you want to solve business challenges through innovative technology? Do you enjoy working on cutting-edge, scalable services technology in a team environment? Do you like working on industry-defining projects that move the needle?At Amazon, we hire the best minds in technology to innovate and build on behalf of our customers. The intense focus we have on our customers is why we are one of the world’s most beloved brands – customer obsession is part of our company DNA. Our Software Development Engineers (SDEs) use cutting-edge technology to solve complex problems and get to see the impact of their work first-hand.The challenges SDEs solve for at Amazon are big and impact millions of customers, sellers, and products around the world. We’re looking for individuals who are excited by the idea of creating new products, features, and services from scratch while managing ambiguity and the pace of a company whose ship c

## for lazy loading 
https://stackoverflow.com/questions/40777864/retrieving-all-information-from-page-beautifulsoup